Import Libraries

In [1]:
import datetime
import mysql.connector
import pandas as pd

Connect to DB

In [2]:
cnx = mysql.connector.connect(
    host='sql11.freemysqlhosting.net',
    user='sql11414637',
    password='C4EIV4fmVq',
    database='sql11414637'
)
cursor = cnx.cursor()

def execute_query(cursor, params):
    cursor.execute(query, params)
    sql_data = pd.DataFrame(cursor.fetchall())

    if len(sql_data) > 0:
        sql_data.columns = cursor.column_names
    else: sql_data = "NO DATA FOUND"

    return sql_data

1. Per uno specifico cliente, determinare le quantità di ciascun prodotto acquistato in un determinato periodo;

In [3]:
query = '''
SELECT p.descrizione, sum(sip.quantita) AS quantita
FROM prodotto p
INNER JOIN spesa_include_prodotto sip ON p.codice = sip.prodotto
INNER JOIN spesa s ON sip.spesa = s.n_scontrino
INNER JOIN cliente c ON s.cliente = c.cf
WHERE c.cf = %s AND s.data BETWEEN %s AND %s
GROUP BY p.codice
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020
# CF DISPONIBILI NEL DB: VTLB4CB4GEI92KW5, 0T1B0ZJKYSZJ32UC, EHLADQRKDFV78NDS, W785D1D4CH8CEI89, WNE850TY9X99MTRD

cf = 'VTLB4CB4GEI92KW5'
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)

execute_query(cursor, (cf, start_date, end_date))

,descrizione,quantita
0,Biscotto Grancereale,2
1,Biscotto Doria,1
2,Fesa di tacchino,2
3,Prosciutto,2
4,Lenticchia,3
5,Fusilli Barilla,3
6,Rigatoni Patti,3


2. Per un determinato prodotto, determinare il numero dei clienti distinti che lo hanno acquistato in un determinato periodo;

In [4]:
query = '''
SELECT p.codice, p.descrizione, count(DISTINCT c.cf) AS n_clienti
FROM prodotto p
INNER JOIN spesa_include_prodotto sip ON p.codice = sip.prodotto
INNER JOIN spesa s ON sip.spesa = s.n_scontrino
INNER JOIN cliente c ON s.cliente = c.cf
WHERE p.codice = %s AND s.data BETWEEN %s AND %s
GROUP BY p.codice
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020
# CODICI PRODOTTI DISPONIBILI NEL DB: 1, 2, 3, 4, 5, 6, 7

cod_prodotto = 1
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)

execute_query(cursor, (cod_prodotto, start_date, end_date))

,codice,descrizione,n_clienti
0,1,Biscotto Grancereale,4


3. Individuare tutti i clienti che hanno acquistato un prodotto in promozione, indicandone il CAP di residenza;

In [5]:
query = '''
SELECT DISTINCT c.*
FROM cliente c
INNER JOIN spesa s on c.cf = s.cliente
INNER JOIN spesa_include_prodotto sip ON s.n_scontrino = sip.spesa
INNER JOIN prodotto p ON p.codice = sip.prodotto
INNER JOIN promozione promo ON promo.prodotto = p.codice
WHERE s.data BETWEEN promo.data_inizio AND promo.data_fine AND c.CAP = %s
'''
# CAP DISPONIBILI NEL DB: 95545, 40265, 30524, 74520, 20085

cap = 95545

execute_query(cursor, (cap,))


,cf,nome,anno_nascita,sesso,reddito,CAP,via,civico,citta
0,0T1B0ZJKYSZJ32UC,Giuseppe Verdi,1985-08-06,M,524415,95545,Via plutarco,2,8


4. Per uno specifico CAP, individuare i clienti che hanno fatto acquisti in un determinato periodo;

In [6]:
query = '''
SELECT DISTINCT c.*
FROM prodotto p
INNER JOIN spesa_include_prodotto sip ON p.codice = sip.prodotto
INNER JOIN spesa s ON sip.spesa = s.n_scontrino
INNER JOIN cliente c ON s.cliente = c.cf
WHERE c.CAP = %s AND s.data BETWEEN %s AND %s
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020
# CAP DISPONIBILI NEL DB: 95545, 40265, 30524, 74520, 20085

cap = 95545
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)

execute_query(cursor, (cap, start_date, end_date))

,cf,nome,anno_nascita,sesso,reddito,CAP,via,civico,citta
0,0T1B0ZJKYSZJ32UC,Giuseppe Verdi,1985-08-06,M,524415,95545,Via plutarco,2,8


5. Per uno specifico CAP, individuare i clienti che non hanno fatto acquisti in un determinato periodo;

In [7]:
query = '''
SELECT c.*
FROM cliente c
WHERE c.CAP = %s AND NOT EXISTS(
	SELECT 1
	FROM prodotto p
	INNER JOIN spesa_include_prodotto sip ON p.codice = sip.prodotto
	INNER JOIN spesa s ON sip.spesa = s.n_scontrino
	INNER JOIN cliente c1 ON s.cliente = c1.cf
	WHERE s.data BETWEEN %s AND %s
)
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020
# CAP DISPONIBILI NEL DB: 95545, 40265, 30524, 74520, 20085

cap = 95545
start_date = datetime.date(2020, 12, 5)
end_date = datetime.date(2020, 12, 6)

execute_query(cursor, (cap, start_date, end_date))

,cf,nome,anno_nascita,sesso,reddito,CAP,via,civico,citta
0,0T1B0ZJKYSZJ32UC,Giuseppe Verdi,1985-08-06,M,524415,95545,Via plutarco,2,8


6. Per ciascun prodotto in promozione determinarne la quantità venduta in un determinato periodo;

In [8]:
query = '''
SELECT p.codice, p.descrizione, sum(sip.quantita) AS quantita
FROM prodotto p
INNER JOIN promozione promo ON promo.prodotto = p.codice
INNER JOIN spesa_include_prodotto sip ON sip.prodotto = p.codice
INNER JOIN spesa s ON sip.spesa = s.n_scontrino
WHERE s.data BETWEEN promo.data_inizio AND promo.data_fine
      AND s.data BETWEEN %s AND %s
GROUP BY p.codice
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020

start_date = datetime.date(2020, 2, 5)
end_date = datetime.date(2020, 4, 5)

execute_query(cursor, (start_date, end_date))

,codice,descrizione,quantita
0,1,Biscotto Grancereale,1
1,2,Biscotto Doria,1
2,3,Fesa di tacchino,1
3,5,Lenticchia,2
4,6,Fusilli Barilla,2


7. I clienti che non hanno mai pagato con bancomat o carta di credito;

In [9]:
query = '''
SELECT c.*
FROM cliente c
WHERE c.cf NOT IN (
	SELECT c1.cf
	FROM cliente c1
	INNER JOIN spesa s ON s.cliente = c1.cf
	INNER JOIN pagamento p ON p.id = s.pagamento
	WHERE p.tipo IN ('bancomat', 'carta')
)
'''

execute_query(cursor, ())

,cf,nome,anno_nascita,sesso,reddito,CAP,via,civico,citta
0,0T1B0ZJKYSZJ32UC,Giuseppe Verdi,1985-08-06,M,524415,95545,Via plutarco,2,8


8. PRIMA VERSIONE - Il cliente che ha speso di più in un determinato periodo;

In [10]:
query = '''
SELECT c.cf, sum(p.totale) AS spesa_totale
FROM cliente c
INNER JOIN spesa s ON c.cf = s.cliente
INNER JOIN pagamento p ON s.pagamento = p.id
WHERE s.data BETWEEN %s AND %s
GROUP BY c.cf
ORDER BY spesa_totale DESC
LIMIT 1
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020

start_date = datetime.date(2020, 12, 4)
end_date = datetime.date(2020, 12, 31)

execute_query(cursor, (start_date, end_date))

,cf,spesa_totale
0,WNE850TY9X99MTRD,0.7


8. SECONDA VERSIONE - Il cliente che ha speso di più in un determinato periodo;

In [11]:
query = '''
SELECT c.cf, sum(p.totale) AS spesa_totale
FROM cliente c
INNER JOIN spesa s ON c.cf = s.cliente
INNER JOIN pagamento p ON s.pagamento = p.id
WHERE s.data BETWEEN %s AND %s
GROUP BY c.cf
HAVING SUM(p.totale) >= ALL(
	SELECT SUM(p2.totale)
	FROM cliente c2
	INNER JOIN spesa s2 ON c2.cf = s2.cliente
	INNER JOIN pagamento p2 ON s2.pagamento = p2.id
	WHERE s2.data BETWEEN %s AND %s
	GROUP BY c2.cf
)
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020

start_date = datetime.date(2020, 12, 4)
end_date = datetime.date(2020, 12, 31)

execute_query(cursor, (start_date, end_date, start_date, end_date))

,cf,spesa_totale
0,WNE850TY9X99MTRD,0.7


9. Per uno specifico CAP la spesa media per scontrino in un determinato periodo;

Dalla rilettura del testo, ho compreso che chiedeva la media della spesa di un prodotto in tutti gli scontrini.
Tuttavia questa è la query che abbiamo chiarito con il professore La Cascia via mail.

In [12]:
query = '''
SELECT c.CAP, AVG(p.totale) as media
FROM cliente c
INNER JOIN spesa s ON c.cf = s.cliente
INNER JOIN pagamento p ON s.pagamento = p.id
WHERE c.CAP = %s AND s.data BETWEEN %s AND %s
GROUP BY c.cf
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020
# CAP DISPONIBILI NEL DB: 95545, 40265, 30524, 74520, 20085

cap = 95545
start_date = datetime.date(2020, 1, 5)
end_date = datetime.date(2020, 12, 6)

execute_query(cursor, (cap, start_date, end_date))

,CAP,media
0,95545,8.798333


9. Per uno specifico CAP la spesa media per scontrino in un determinato periodo;

Questa invece è la versione che ho inteso dopo: "la media della spesa di un prodotto in tutti gli scontrini"

In [13]:
query = '''
SELECT p.codice, p.descrizione, AVG(sip.prezzo) as media
FROM cliente c
INNER JOIN spesa s ON c.cf = s.cliente
INNER JOIN spesa_include_prodotto sip on sip.spesa = s.n_scontrino
INNER JOIN prodotto p on p.codice = sip.prodotto
WHERE c.CAP = %s AND s.data BETWEEN %s AND %s
GROUP BY p.codice
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020
# CAP DISPONIBILI NEL DB: 95545, 40265, 30524, 74520, 20085

cap = 95545
start_date = datetime.date(2020, 1, 5)
end_date = datetime.date(2020, 12, 6)

execute_query(cursor, (cap, start_date, end_date))

,codice,descrizione,media
0,1,Biscotto Grancereale,0.641667
1,2,Biscotto Doria,0.760000
2,3,Fesa di tacchino,4.920000
3,4,Prosciutto,6.000000
4,5,Lenticchia,0.540000
5,6,Fusilli Barilla,1.425000
6,7,Rigatoni Patti,0.700000


10. Per ciascuna categoria di prodotti il totale del prezzo pagato per i prodotti venduti in un determinato periodo.

In [14]:
query = '''
SELECT cat.nome, sum(sip.prezzo) as totale_pagato
FROM prodotto p
INNER JOIN categoria cat ON p.categoria = cat.id
INNER JOIN spesa_include_prodotto sip ON p.codice = sip.prodotto
INNER JOIN spesa s ON sip.spesa = s.n_scontrino
INNER JOIN cliente c ON s.cliente = c.cf
WHERE s.data BETWEEN %s AND %s
GROUP BY cat.id
'''
# DATE DISPONIBILI NEL DB: Dal 2020-01-01 al 31-12-2020

start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2020, 12, 31)

execute_query(cursor, (start_date, end_date))

,nome,totale_pagato
0,biscotti,14.425000
1,salumi,76.800001
2,legumi,4.320000
3,pasta,21.924999


Close connection

In [15]:
cursor.close()
cnx.close()